In [2]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,process_datesIT

In [3]:
#get list of tickers (just dow until everything is set up)
dow_url = "https://financialmodelingprep.com/api/v3/dowjones_constituent?apikey=600e501fe324b2f1cbb2aec5b390b6db"
dow_tick = pd.read_json(dow_url)
tickers = list(dow_tick.symbol.values)
print(tickers,)
tickers += ['SECO']

['CRM', 'WBA', 'V', 'NKE', 'UNH', 'TRV', 'VZ', 'INTC', 'WMT', 'JNJ', 'DIS', 'MCD', 'JPM', 'CAT', 'BA', 'AMGN', 'DOW', 'AAPL', 'GS', 'CSCO', 'MSFT', 'HD', 'PG', 'MRK', 'IBM', 'HON', 'KO', 'CVX', 'AXP', 'MMM']


In [3]:
indexes = None
etfs = None

In [4]:
! rm ../../Finances.db
! rm schema.sql

In [5]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [6]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()

In [7]:
l = []

In [8]:
tickers = ['AAPL']

In [11]:
%%time
schemas = open("schema.sql", "a+")
for name, url in zip(urls.index[28:],urls.values[28:]):
    print(name)
    if url =='NONE':
            continue
    if type(url) != dict:
        #extract
        if name[0]=='G': #if ticker info not needed
            tab = get_appended(url,apikey,name,verbose=True)
        elif name[0] == 'S' or name[0] == 'T':
            tab = get_appended(url,apikey,name,tickers=tickers,verbose=True)
        #transform
        tab.reset_index(inplace=True,drop=True)
        tab.drop_duplicates(inplace=True)
        #load
        l.append(tab)
        a = pd.io.sql.get_schema(tab,name)
        c.execute('drop table if exists '+ name)
        c.execute(a)
        tosql = a +';\n\n'
        schemas.write(tosql)
        tab.to_sql(name,con=conn,index=False,if_exists='append')
    else: 
        for i, u in enumerate(url.values()):
            print(u)
            if i == 1: 
                #extract
                tab = get_appended(u,apikey,tickers,verbose=True)
                #transform
                tab.drop_duplicates(inplace=True)
                #load
                a = pd.io.sql.get_schema(tab,name)
                c.execute('drop table if exists '+ name)
                c.execute(a)
                tosql = a +';\n\n'
                schemas.write(tosql)
                tab.to_sql(name,con=conn,index=False,if_exists='append')
                print(c.execute('select * from '+name+' limit 1;').fetchall())
            else:
                tab= get_appended(u,apikey,tickers,verbose=True)
                tab.drop_duplicates(inplace=True)
                tab.to_sql(name,con=conn,index=False,if_exists='append')

S2_CV_DCF_YR


TypeError: get_appended() got multiple values for argument 'tickers'

In [10]:
a = get_appended(url,apikey,tickers=['AAPL'],json=3,verbose=True)

AttributeError: 'dict' object has no attribute 'replace'

In [ ]:
a1 = a.copy()

In [ ]:
a2 = a1.iloc[:,1].apply(lambda x: pd.Series(x))

In [24]:
a1

,symbol,historical
0,AAPL,"{'date': '2021-02-25', 'open': 124.68, 'high':..."
1,AAPL,"{'date': '2021-02-24', 'open': 124.940002, 'hi..."
2,AAPL,"{'date': '2021-02-23', 'open': 123.760002, 'hi..."
3,AAPL,"{'date': '2021-02-22', 'open': 128.009995, 'hi..."
4,AAPL,"{'date': '2021-02-19', 'open': 130.240005, 'hi..."
...,...,...
10131,AAPL,"{'date': '1980-12-18', 'open': 0.118862, 'high..."
10132,AAPL,"{'date': '1980-12-17', 'open': 0.1155129999999..."
10133,AAPL,"{'date': '1980-12-16', 'open': 0.1132809999999..."
10134,AAPL,"{'date': '1980-12-15', 'open': 0.1222100000000..."
